### Split and merge different MRI image modalities.


**Split:**
- Original folder: t1, t1Gd, t2, flair, segmentation;
- After split(example 1): t1, t1Gd, t2 and flair in one folder; segmentation in another folder;
- After split(example 2): t1, t1Gd, t2, flair and segmentation each one in a folder, in total 5 folders;


In [ ]:
import os
import shutil
from mySettings import get_split_folder_setting_dict

import sys
sys.path.append("E://2020_MRI_Work/HarmonizationProject")
from utils.myUtils import traversalDir_FirstDir, mkdir, get_filenames

In [ ]:
def main_split_different_modalities(split_folder_setting):
    image_basepath=split_folder_setting["image_folder"]
    save_basepath=split_folder_setting["save_basepath"]
    split_criterion_dict=split_folder_setting["split_criterion"]
    used_to_find_basename=split_folder_setting["used_to_find_basename"]
    total_number_of_modalities=split_folder_setting["total_number_of_modalities"]
    
    patient_id_list=traversalDir_FirstDir(image_basepath)
    num_of_patients=len(patient_id_list)
    i=0
    for patient_id in patient_id_list:
        i=i+1
        print("\n-{}/{}:{}".format(i, num_of_patients, patient_id))
        original_patient_folder=os.path.join(image_basepath, patient_id)
        
        original_patient_files=get_filenames(original_patient_folder)
        if total_number_of_modalities==len(original_patient_files):
            #find the patient basename for this patient
            for orginal_patient_file in original_patient_files:
                if orginal_patient_file.endswith("_"+used_to_find_basename+".nii.gz"):
                    patient_basename=(os.path.basename(orginal_patient_file)).replace("_"+used_to_find_basename+".nii.gz", "")

            #split the folder for this patient
            for split_name, split_criterion in split_criterion_dict.items():
                if split_name=="segmentation" and len(split_criterion)==1:
                    modality=split_criterion[0]
                    patient_split_folder=os.path.join(save_basepath, split_name)
                    mkdir(patient_split_folder)
                    src_image_path=os.path.join(original_patient_folder, patient_basename+"_"+modality+".nii.gz")
                    dest_image_path=os.path.join(patient_split_folder, patient_id+".nii.gz")
                    shutil.copy(src_image_path, dest_image_path) 
                else:
                    patient_split_folder=os.path.join(save_basepath, split_name, patient_id)
                    mkdir(patient_split_folder)
                    for modality in split_criterion:
                        src_image_path=os.path.join(original_patient_folder, patient_basename+"_"+modality+".nii.gz")
                        dest_image_path=os.path.join(patient_split_folder, patient_id+"_"+modality+".nii.gz")
                        shutil.copy(src_image_path, dest_image_path) 
        else:
            print("\n-Only {} modalities for patient {}. \n{}".format(len(original_patient_files), patient_id, original_patient_files))
                
    
            

### Main: split different image modalities and the segmentation mask into different folders.

In [ ]:
split_folder_setting_dict=get_split_folder_setting_dict()

for setting_name, split_folder_setting in split_folder_setting_dict.items():
    print("\n===== Split folders for {} .====".format(setting_name))
    main_split_different_modalities(split_folder_setting)
    print("Finish splitting folders for {}.".format(setting_name))